# Construction des BDD pour le Machine Learning/régression

Dans ce notebook, nous allons encoder les variables catégorielles des bdd à l'aide du one hot encoding pour pouvoir utiliser les variables catégorielles dans les régressions.

In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

c:\Users\elelo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
enedis_IDF_6=pd.read_csv("Données/data_conso_moyenne/data_conso_6kva_IDF.csv")
enedis_HDF_6=pd.read_csv("Données/data_conso_moyenne/data_conso_6kva_HDF.csv")
enedis_PACA_6=pd.read_csv("Données/data_conso_moyenne/data_conso_6kva_PACA.csv")

## 1. Construction de la bdd pour "Jour" :

In [14]:
def creation_bdd_one_hot_jour(bdd_region,nom_court,kva):
    # Création d'un ensemble de données fictif avec une variable catégorielle
    Xdate =bdd_region["date"].to_numpy()
    Xdate=Xdate.reshape(-1, 1)

    # Création de l'objet OneHotEncoder
    encoder = OneHotEncoder()

    # Encodage des données avec OneHotEncoder
    X_encoded = encoder.fit_transform(Xdate)
    oneHot=pd.DataFrame(X_encoded.toarray(),index=bdd_region.index,columns=["jour_"+str(i+1) for i in range(len(X_encoded.toarray()))])
    oneHot["indice"]=oneHot.index
    bdd_region["indice"]=bdd_region.index
    bdd_region=bdd_region.merge(oneHot,on="indice")
    bdd_region.to_csv("Données/data_one_hot_encoded/data_one_hot_encoded_jour_"+kva+"_"+nom_court+".csv",index=False)

In [15]:
creation_bdd_one_hot_jour(enedis_IDF_6,"IDF",'6kva')
creation_bdd_one_hot_jour(enedis_HDF_6,"HDF",'6kva')
creation_bdd_one_hot_jour(enedis_PACA_6,"PACA",'6kva')

## 2. Construction de la bdd pour "heure par 30min" et "mois" :

On construit d'abord la bdd de conso moyenne par 30 min :

In [24]:
def bdd_30min(enedis_final,region):    
    conso_region=enedis_final[enedis_final["region_x"]==region]
    temperature=enedis_final[["date","region_x","tmoy"]]
    temperature_region=temperature[temperature.region_x==region]
    temperature_region.drop_duplicates(inplace=True)


    conso_region["heure"]=enedis_final["horodate"].apply(lambda x : x[11:16])

    nb_contrat=conso_region.nb_points_soutirage.mean()

    total_conso_region=conso_region.groupby(["date","heure"])["total_energie_soutiree_wh"].sum().reset_index() # pour chaque jour, on calcule la conso totale pour chaque heure
    total_conso_region["mois"]=total_conso_region["date"].apply(lambda x : x[5:7])
    total_conso_region["res_conso_moyenne"]=total_conso_region.total_energie_soutiree_wh/nb_contrat # conso moyenne soutirée par contrat pour chaque 30min

    total_conso_region=total_conso_region.merge(temperature_region,on="date",how="right")

    return(total_conso_region)

In [34]:
enedis_final_6kva=pd.read_csv("Données/enedis_filtered_6kva.csv",sep=",")

In [35]:
enedis_IDF_30=bdd_30min(enedis_final_6kva,"Île-de-France")
enedis_HDF_30=bdd_30min(enedis_final_6kva,"Hauts-de-France")
enedis_PACA_30=bdd_30min(enedis_final_6kva,"Provence-Alpes-Côte d'Azur")

c:\Users\elelo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\elelo\AppData\Local\Temp/ipykernel_3380/3971548881.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conso_region["heure"]=enedis_final["horodate"].apply(lambda x : x[11:16])


In [41]:
def creation_bdd_one_hot_heure_mois(bdd_region,nom_court,kva):
    # Création d'un ensemble de données fictif avec une variable catégorielle
    Xheure =bdd_region["heure"].to_numpy()
    Xheure=Xheure.reshape(-1, 1)

    Xmois =bdd_region["mois"].to_numpy()
    Xmois=Xmois.reshape(-1, 1)

    # Création de l'objet OneHotEncoder
    encoder = OneHotEncoder()

    # Encodage des données avec OneHotEncoder
    X_encoded_heure = encoder.fit_transform(Xheure)

    oneHot_heure=pd.DataFrame(X_encoded_heure.toarray(),index=bdd_region.index,columns=["heure_"+str(i+1) for i in range(X_encoded_heure.shape[1])])
    oneHot_heure["indice"]=oneHot_heure.index
    bdd_region["indice"]=bdd_region.index
    bdd_region=bdd_region.merge(oneHot_heure,on="indice")
    

    X_encoded_mois = encoder.fit_transform(Xmois)

    oneHot_mois=pd.DataFrame(X_encoded_mois.toarray(),index=bdd_region.index,columns=["mois_"+str(i+1) for i in range(X_encoded_mois.shape[1])])
    oneHot_mois["indice"]=oneHot_mois.index

    bdd_region=bdd_region.merge(oneHot_mois,on="indice")
    bdd_region.to_csv("Données/data_one_hot_encoded/data_one_hot_encoded_heure_mois_"+kva+"_"+nom_court+".csv",index=False)


In [42]:
creation_bdd_one_hot_heure_mois(enedis_IDF_30,"IDF",'6kva')
creation_bdd_one_hot_heure_mois(enedis_HDF_30,"HDF",'6kva')
creation_bdd_one_hot_heure_mois(enedis_PACA_30,"PACA",'6kva')